# Part 3 - Text analysis

# 3.a Computing PMI

In this assessment you are tasked to discover strong associations between concepts in Airbnb reviews. The starter code we provide in this notebook is for orientation only. The below imports are enough to implement a valid answer.

### Imports, data loading and helper functions

We first connect our google drive, import pandas, numpy and some useful nltk and collections modules, then load the dataframe and define a function for printing the current time, useful to log our progress in some of the tasks.

In [ ]:
import pandas as pd
from nltk.tag import pos_tag
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
import string
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# load stopwords
sw = set(stopwords.words('english'))

In [ ]:
p = ""
df = pd.read_csv(os.path.join(p,'reviews.csv'))
# deal with empty reviews
df.comments = df.comments.fillna('')

In [ ]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...


In [ ]:
df.shape

(452143, 6)

### 3.a1 - Process reviews

What to implement: A `function process_reviews(df)` that will take as input the original dataframe and will return it with three additional columns: `tokenized`, `tagged` and `lower_tagged`.

In [ ]:
def process_reviews(df):
  '''
  1- tokenization of reviews save it under tokenized column

  2-  divide senteces as words, remove punctuation and post tag all of them

  3- take all words in tagged and make them lower

  return:
    dataframe with new columns
  '''
  tokenized = df.comments.apply(sent_tokenize) # tokenization of reviews 
  df["tokenized"] = tokenized

  tagged = []

  for comment in df.comments:
    tagged.append(pos_tag(comment.translate(str.maketrans('', '', string.punctuation)).split())) # divide senteces as words, remove punctuation and post tag all of them

  df["tagged"] = tagged

  lower_tagged = []

  for tag in df.tagged:   # take all words in tagged and make them lower
    arr = []
    for word in tag:
      wrd = (word[0].lower(), word[1])
      arr.append(wrd)
    lower_tagged.append(arr) 

  df["lower_tagged"] = lower_tagged

  return df

In [ ]:
df = process_reviews(df)

In [ ]:
df.head(10)

,listing_id,id,date,reviewer_id,reviewer_name,comments,tokenized,tagged,lower_tagged
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...,"[Daniel is really cool., The place was nice an...","[(Daniel, NNP), (is, VBZ), (really, RB), (cool...","[(daniel, NNP), (is, VBZ), (really, RB), (cool..."
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...,"[Daniel is the most amazing host!, His place i...","[(Daniel, NNP), (is, VBZ), (the, DT), (most, R...","[(daniel, NNP), (is, VBZ), (the, DT), (most, R..."
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,"[We had such a great time in Amsterdam., Danie...","[(We, PRP), (had, VBD), (such, JJ), (a, DT), (...","[(we, PRP), (had, VBD), (such, JJ), (a, DT), (..."
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...,"[Very professional operation., Room is very cl...","[(Very, RB), (professional, JJ), (operation, N...","[(very, RB), (professional, JJ), (operation, N..."
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...,"[Daniel is highly recommended., He provided al...","[(Daniel, NNP), (is, VBZ), (highly, RB), (reco...","[(daniel, NNP), (is, VBZ), (highly, RB), (reco..."
5,2818,4748,2009-06-29,20192,Jie,Daniel was a great host! He made everything so...,"[Daniel was a great host!, He made everything ...","[(Daniel, NNP), (was, VBD), (a, DT), (great, J...","[(daniel, NNP), (was, VBD), (a, DT), (great, J..."
6,2818,5202,2009-07-07,23055,Vanessa,Daniele is an amazing host! He provided everyt...,"[Daniele is an amazing host!, He provided ever...","[(Daniele, NNP), (is, VBZ), (an, DT), (amazing...","[(daniele, NNP), (is, VBZ), (an, DT), (amazing..."
7,2818,9131,2009-09-06,26343,Katja,You can´t have a nicer start in Amsterdam. Dan...,"[You can´t have a nicer start in Amsterdam., D...","[(You, PRP), (can´t, VBP), (have, VB), (a, DT)...","[(you, PRP), (can´t, VBP), (have, VB), (a, DT)..."
8,2818,12103,2009-10-01,40999,Marie-Eve,Daniel was a fantastic host. His place is calm...,"[Daniel was a fantastic host., His place is ca...","[(Daniel, NNP), (was, VBD), (a, DT), (fantasti...","[(daniel, NNP), (was, VBD), (a, DT), (fantasti..."
9,2818,16196,2009-11-04,38623,Graham,Daniel was great. He couldn.t do enough for us...,"[Daniel was great., He couldn.t do enough for ...","[(Daniel, NNP), (was, VBD), (great, JJ), (He, ...","[(daniel, NNP), (was, VBD), (great, JJ), (he, ..."


### 3.a2 - Create a vocabulary

What to implement: A function `get_vocab(df)` which takes as input the DataFrame generated in step 1.c, and returns two lists, one for the 1,000 most frequent center words (nouns) and one for the 1,000 most frequent context words (either verbs or adjectives). 

In [ ]:
def get_vocab(df):
  '''
  take lower than one bu one and put all in an array, calculate frequency of each word, take most common words
  if tag start with N, it is a center word, if tag start with J or N, it is a context word

  take most used 1000 center word, take most used 1000 context word

  return:
      most used 1000 center word and most used 1000 context word
  

  '''
  vocab = []

  for sentence in df.lower_tagged:  #  take lower tag one bu one and put all in an array
    vocab.extend(sentence)
  
  freq = Counter(vocab)  # calculate frequency of each word

  most_common_words = freq.most_common() # take most common words

  

  most_common_nouns_arr = []     # if tag start with N, it is a center word
  for i in most_common_words:
    if i[0][1][0] == "N":
      most_common_nouns_arr.append(i)

  most_common_context = []    # if tag start with J or N, it is a context word
  for i in most_common_words:
    if i[0][1][0] == "J" or i[0][1][0] == "V":
      most_common_context.append(i)

  center = []
  for i in most_common_nouns_arr:  # take most used 1000 center word
    center.append(i[0][0])
    if len(list(set(center))) == 1000:
      break

  cent_vocab = list(set(center))
  

  context = []
  for i in most_common_context:  # take most used 1000 context word
    context.append(i[0][0])
    if len(list(set(context))) == 1000:
      break

  cont_vocab = list(set(context))




  return cent_vocab, cont_vocab

In [ ]:
cent_vocab, cont_vocab = get_vocab(df)

### 3.a3 Count co-occurrences between center and context words

What to implement: A function `get_coocs(df, center_vocab, context_vocab)` which takes as input the DataFrame generated in step 1, and the lists generated in step 2 and returns a dictionary of dictionaries, of the form in the example above. It is up to you how you define context (full review? per sentence? a sliding window of fixed size?), and how to deal with exceptional cases (center words occurring more than once, center and context words being part of your vocabulary because they are frequent both as a noun and as a verb, etc). Use comments in your code to justify your approach. 

In [ ]:
df.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments,tokenized,tagged,lower_tagged
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...,"[Daniel is really cool., The place was nice an...","[(Daniel, NNP), (is, VBZ), (really, RB), (cool...","[(daniel, NNP), (is, VBZ), (really, RB), (cool..."
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...,"[Daniel is the most amazing host!, His place i...","[(Daniel, NNP), (is, VBZ), (the, DT), (most, R...","[(daniel, NNP), (is, VBZ), (the, DT), (most, R..."
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,"[We had such a great time in Amsterdam., Danie...","[(We, PRP), (had, VBD), (such, JJ), (a, DT), (...","[(we, PRP), (had, VBD), (such, JJ), (a, DT), (..."
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...,"[Very professional operation., Room is very cl...","[(Very, RB), (professional, JJ), (operation, N...","[(very, RB), (professional, JJ), (operation, N..."
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...,"[Daniel is highly recommended., He provided al...","[(Daniel, NNP), (is, VBZ), (highly, RB), (reco...","[(daniel, NNP), (is, VBZ), (highly, RB), (reco..."


In [ ]:
df_test = df.head(10)

In [ ]:
def get_coos(df, cent_vocab, cont_vocab):

  sentences = []
  for sentence in df.tokenized:
    for i in sentence:
     sentences.extend([i.translate(str.maketrans('', '', string.punctuation)).lower()]) # take all sentences one by one and remove all punctuations and also make all sentences lower
 
  coocs = {}   # create coocs dict, we will return this as a result

  for sentence in sentences:  # take all sentences
    for vocab in cent_vocab:  # take all senter words
      dictt = {}
      if vocab in sentence.split():   # split sentences to make it list and search center words in this list 
        for cont in cont_vocab:    # if there is center words in the list then now search for context words
          if cont in  sentence.split():
            number = sentence.count(cont) #. if context words have found, count how many ?
            dictt.update({cont:number})   # create dict key: context word name, value : number of occurance

        if vocab in list(coocs.keys()):  # check is there dictionary before with same key if there is append disctionaries
          dict_previous = coocs.get(vocab)
          coocs.update({vocab: {k: dict_previous.get(k, 0) + dictt.get(k, 0) for k in set(dict_previous) | set(dictt)}})
        else:
          coocs.update({vocab:dictt})   # put the dict above inside of coocs list as a value, the key is center word
  return coocs

In [ ]:
len(df)

452143

In [ ]:
import time
start = time.process_time()
coocs = get_coos(df, cent_vocab, cont_vocab)
print(time.process_time() - start)


11475.934205631


### 3.a4 Convert co-occurrence dictionary to 1000x1000 dataframe
What to implement: A function called `cooc_dict2df(cooc_dict)`, which takes as input the dictionary of dictionaries generated in step 3 and returns a DataFrame where each row corresponds to one center word, and each column corresponds to one context word, and cells are their corresponding co-occurrence value. Some (x,y) pairs will never co-occur, you should have a 0 value for those cases. 

In [ ]:
def cooc_dict2df(coocs):
  '''
  1- take all keys and they are center words

  2- take all keys in values of coocs and they are context words

  3- create matrix with center words as an index, context words as feature

  4- in coocs dict, fill all values in the dataframe one by one

  return: 
  1000 * 1000 occurance matrix
  '''

  center_w = list(coocs.keys()) # take all keys and they are center words

  context_w = set()
  for keys, values in coocs.items():  # # take all keys in values of coocs and they are context words
    for k,v in values.items():
      context_w.add(k)


  coocdf = pd.DataFrame(0, index= center_w, columns= context_w)   # create matrix with center words as an index, context words as feature
  for keys, values in coocs.items():  # in coocs dict, fill all values in the dataframe one by one
    for k, v in values.items():
      coocdf.at[keys, k] = v
  return coocdf

In [ ]:
coocdf = cooc_dict2df(coocs)
coocdf.shape

(1000, 1000)

In [ ]:
coocdf.to_csv("coocdf.csv")

In [ ]:
coocdf

,sleep,petite,recommend,climbing,answered,aber,natural,needed,shower,vélos,se,liegt,looking,checking,end,justice,dans,shared,coming,hard,u,based,doing,longer,nicest,white,suitable,said,centre,uns,love,cosy,little,séjour,say,nous,bad,compact,typical,mais,...,ready,calm,taking,told,saying,dining,cool,ran,pretty,neighborhood,deal,loved,try,cual,chambre,playing,4th,miss,wellequipped,arrived,particular,show,returning,woke,everything,uncomfortable,chez,found,works,recomend,european,eine,accueil,seemed,detail,think,nest,late,suggested,thats
daniel,2,1,67,0,10,3,1,47,6,1,11,0,4,4,4,0,9,5,2,2,11,1,1,1,1,0,0,4,9,24,5,9,18,16,8,58,0,0,1,5,...,6,0,2,2,1,0,9,1,3,11,6,15,4,1,1,0,0,0,2,41,1,6,2,1,80,2,19,8,3,1,2,7,5,1,3,4,1,13,3,3
really,363,2,3688,14,248,0,73,1105,631,0,3,0,374,133,290,143,1,167,190,148,202,28,62,115,27,26,82,109,1790,0,523,1450,1448,0,341,3,220,14,104,0,...,169,446,128,96,13,70,1766,16,513,1641,135,1315,119,0,0,30,18,65,60,645,35,127,31,15,4178,44,2,380,76,77,16,0,1,88,128,247,15,509,54,148
cool,46,2,195,3,3,3,18,79,94,4,22,0,92,10,35,6,31,8,25,11,40,7,5,6,7,8,2,14,244,5,33,131,280,14,21,40,9,0,26,16,...,16,56,11,19,2,14,9528,3,225,673,7,129,17,1,7,4,1,9,4,34,6,13,2,0,404,4,4,35,12,6,3,4,9,12,7,21,2,37,14,41
was,1568,4,4339,78,1468,293,333,11511,3780,0,14,60,1397,1353,1467,145,7,993,890,735,600,149,298,559,190,133,182,817,4748,511,870,3360,6069,2,1316,2,821,73,291,1,...,1404,722,543,991,129,364,2000,178,1998,5060,791,3223,337,0,1,135,124,161,290,6475,160,886,134,123,29955,332,6,1561,226,56,180,314,1,377,514,847,42,4008,235,289
place,933,25,19239,37,86,0,126,1752,556,28,125,0,3439,141,589,291,136,189,496,249,535,64,60,259,109,40,145,189,2893,1,1517,2421,2647,55,782,287,213,60,173,97,...,225,795,179,169,38,88,1599,23,803,2554,143,3217,327,0,43,27,47,135,82,725,52,286,85,13,6333,46,19,837,91,229,56,0,22,60,123,561,22,392,77,313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
martijn,1,1,31,0,11,0,0,21,2,1,2,0,3,1,4,0,4,1,2,0,1,0,0,0,1,0,0,1,7,8,3,3,14,11,7,26,1,0,1,4,...,8,1,0,4,0,0,5,1,0,5,2,7,1,0,1,0,1,1,0,17,0,1,0,1,41,0,5,2,0,0,0,0,3,1,1,2,0,8,0,0
mirjam,2,1,38,0,2,0,0,25,6,1,11,0,3,3,3,0,10,4,1,1,9,0,1,1,2,0,1,1,6,12,3,6,8,32,3,50,0,0,1,5,...,3,0,0,2,1,0,0,0,5,2,2,3,0,0,12,0,0,0,0,19,0,1,1,0,46,0,22,1,0,0,0,13,13,1,3,4,0,10,2,2
henk,0,3,19,0,2,6,0,16,0,6,9,0,5,0,3,0,8,2,1,1,3,0,0,0,1,0,0,0,8,45,8,4,12,12,5,43,0,0,0,3,...,4,1,5,2,0,1,1,0,3,2,1,12,0,0,1,1,0,0,0,17,0,1,0,0,16,0,12,1,0,0,0,20,2,0,1,1,0,6,0,1
jeroen,0,0,28,0,5,3,0,16,1,3,20,0,3,3,0,0,11,1,1,3,10,0,0,0,0,0,0,1,9,21,2,5,13,19,4,53,0,0,1,9,...,6,1,2,7,0,0,10,0,1,6,3,7,1,0,3,0,0,0,0,23,0,3,0,0,53,0,11,3,0,2,0,2,8,0,1,1,1,10,0,1


### 3.a5 Raw co-occurrences to PMI scores

What to implement: A function `cooc2pmi(df)` that takes as input the DataFrame generated in step 4, and returns a new DataFrame with the same rows and columns, but with PMI scores instead of raw co-occurrence counts. 

In [ ]:
def cooc2pmi(df):
  '''
  at first, find common words in both context and center words. we will make them 0 pmi because they can effect realted word list.

  then calculate the pmi using formula which was given to us in the assignment.

  return:
  matrix with pmi values
  '''

  index_list = coocdf.index.to_list() # take all center_words
  feature_list = coocdf.columns.to_list() # take all context words
  common_words = set(index_list) - (set(index_list) - set(feature_list)) # find comomon words

  for i in common_words:  # to calculate pmi if there is common words in both index and feature make this value 0 because it will effect pmi score
    coocdf.at[i, i] = 0

  positive=True     # CALCULATE PMI
  col_totals = df.sum(axis=0)
  total = col_totals.sum()
  row_totals = df.sum(axis=1)
  expected = np.outer(row_totals, col_totals) / total # divide total to find pmi
  pmidf = df / expected
  
  return pmidf

In [ ]:
pmidf = cooc2pmi(coocdf)
pmidf.shape

(1000, 1000)

In [ ]:
pmidf.head(10)

,sleep,petite,recommend,climbing,answered,aber,natural,needed,shower,vélos,se,liegt,looking,checking,end,justice,dans,shared,coming,hard,u,based,doing,longer,nicest,white,suitable,said,centre,uns,love,cosy,little,séjour,say,nous,bad,compact,typical,mais,...,ready,calm,taking,told,saying,dining,cool,ran,pretty,neighborhood,deal,loved,try,cual,chambre,playing,4th,miss,wellequipped,arrived,particular,show,returning,woke,everything,uncomfortable,chez,found,works,recomend,european,eine,accueil,seemed,detail,think,nest,late,suggested,thats
daniel,0.244641,0.309659,0.875782,0.000000,2.566040,0.440434,0.452841,1.658897,0.414872,0.306798,0.385089,0.000000,0.274325,1.466618,0.507948,0.000000,0.301130,1.177295,0.379674,0.723901,1.485041,1.393833,0.811379,0.307504,0.872769,0.000000,0.000000,1.379768,0.138334,1.473270,0.497850,0.500594,0.538604,0.862582,1.183954,1.074648,0.000000,0.000000,0.320049,0.340186,...,1.531317,0.000000,0.651541,0.630787,1.871066,0.000000,0.713371,1.651950,0.310487,0.304555,2.554748,0.726090,1.315343,0.773456,0.099134,0.000000,0.000000,0.000000,1.618272,2.743181,1.364194,2.054070,1.800915,1.983600,0.978980,1.964463,2.972716,0.901215,1.822997,1.390522,3.019522,0.537505,0.732357,0.745864,1.274477,0.740996,0.434845,1.184395,3.177977,1.055585
really,1.217735,0.016985,1.322086,0.626314,1.745269,0.000000,0.906600,1.069624,1.196573,0.000000,0.002880,0.000000,0.703436,1.337384,1.009961,3.472640,0.000918,1.078398,0.989195,1.469124,0.747901,1.070326,1.379630,0.969829,0.646265,1.029731,1.617820,1.031146,0.754550,0.000000,1.428163,2.211864,1.188263,0.000000,1.384034,0.001524,1.246872,0.600985,0.912846,0.000000,...,1.182900,1.889122,1.143587,0.830369,0.667083,0.668456,3.838935,0.724877,1.456084,1.246031,1.576442,1.745710,1.073182,0.000000,0.000000,1.226344,0.604419,1.383813,1.331435,1.183526,1.309458,1.192381,0.765548,0.816005,1.402166,1.185260,0.008582,1.174004,1.266559,2.936405,0.662484,0.000000,0.004017,1.800071,1.491311,1.254874,0.178885,1.271798,1.568811,1.428174
cool,1.298699,0.142944,0.588313,1.129510,0.177679,0.101656,1.881352,0.643577,1.500177,0.283247,0.177764,0.000000,1.456282,0.846271,1.025839,1.226252,0.239401,0.434768,1.095401,0.918955,1.246401,2.251961,0.936366,0.425847,1.410099,2.666522,0.332086,1.114619,0.865624,0.070842,0.758394,1.681770,1.933780,0.174205,0.717326,0.171061,0.429286,0.000000,1.920623,0.251258,...,0.942510,1.996264,0.827098,1.383116,0.863717,1.125143,0.000000,1.143853,5.374728,4.300709,0.687934,1.441255,1.290268,0.178520,0.160167,1.376119,0.282599,1.612543,0.747022,0.525051,1.889207,1.027210,0.415667,0.000000,1.141083,0.906830,0.144448,0.910036,1.683054,1.925667,1.045398,0.070892,0.304262,2.065822,0.686374,0.897898,0.200732,0.778049,3.423022,3.329722
was,1.338465,0.008644,0.395798,0.887920,2.628765,0.300185,1.052330,2.835287,1.823965,0.000000,0.003420,0.130139,0.668597,3.461921,1.300024,0.895997,0.001634,1.631649,1.179053,1.856517,0.565274,1.449303,1.687339,1.199566,1.157218,1.340346,0.913698,1.966666,0.509285,0.218904,0.604520,1.304201,1.267291,0.000752,1.359137,0.000259,1.184015,0.797395,0.649938,0.000475,...,2.500595,0.778175,1.234452,2.181166,1.684384,0.884487,1.106280,2.052009,1.443044,0.977655,2.350366,1.088734,0.773341,0.000000,0.000692,1.404235,1.059504,0.872177,1.637502,3.023240,1.523205,2.116703,0.842035,1.702634,2.558086,2.275695,0.006551,1.227167,0.958375,0.543411,1.896457,0.168258,0.001022,1.962290,1.523829,1.094968,0.127452,2.548256,1.737239,0.709630
place,1.649184,0.111870,3.634064,0.872182,0.318897,0.000000,0.824527,0.893603,0.555554,0.124137,0.063236,0.000000,3.408211,0.747076,1.080844,3.723558,0.065757,0.643081,1.360665,1.302379,1.043730,1.289079,0.703499,1.150903,1.374721,0.834741,1.507390,0.942099,0.642576,0.000887,2.182746,1.945927,1.144562,0.042848,1.672401,0.076844,0.636092,1.357152,0.800113,0.095369,...,0.829822,1.774327,0.842663,0.770244,1.027452,0.442791,1.831513,0.549052,1.200953,1.021839,0.879876,2.250292,1.553873,0.000000,0.061600,0.581562,0.831582,1.514395,0.958791,

### 3.a6 Retrieve top-k context words, given a center word

What to implement: A function `topk(df, center_word, N=10)` that takes as input: (1) the DataFrame generated in step 5, (2) a `center_word` (a string like `‘towels’`), and (3) an optional named argument called `N` with default value of 10; and returns a list of `N` strings, in order of their PMI score with the `center_word`. You do not need to handle cases for which the word `center_word` is not found in `df`. 

In [ ]:
def topk(df, center_word, N=10):
  '''
  filter the words, select the word that what we want to see. I will be a single row.

  take a transpoze, sort is descending take top N'th word which is related to our selected word

  return:
  top related words
  '''
  top_words = df[df.index == center_word].T.sort_values(center_word, ascending=False).head(N).index.to_list()
  return top_words

In [ ]:
topk(pmidf, 'quick')

['respond',
 'reply',
 'responding',
 'answer',
 'responds',
 'efficient',
 'answering',
 'communicate',
 'ride',
 'thorough']

In [ ]:
topk(pmidf, 'hotel',5)

['compared', 'cheaper', 'decided', 'expensive', 'glad']

In [ ]:
topk(pmidf, 'coffee')

['tea',
 'complimentary',
 'cheese',
 'fresh',
 'stocked',
 'shops',
 'italian',
 'wine',
 'delicious',
 'included']